In [ ]:
import os
import pandas as pd


In [ ]:
os.environ['KAGGLE_CONFIG_DIR'] = '/content'
!kaggle datasets download -d gpreda/covid-world-vaccination-progress

df = pd.read_csv('country_vaccinations.csv')
df.head(10)


covid-world-vaccination-progress.zip: Skipping, found more recently modified local copy (use --force to force download)


,country,iso_code,date,total_vaccinations,people_vaccinated,people_fully_vaccinated,daily_vaccinations_raw,daily_vaccinations,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,daily_vaccinations_per_million,vaccines,source_name,source_website
0,Afghanistan,AFG,2021-02-22,0.0,0.0,NaN,NaN,NaN,0.00,0.00,NaN,NaN,Oxford/AstraZeneca,Government of Afghanistan,http://www.xinhuanet.com/english/asiapacific/2...
1,Afghanistan,AFG,2021-02-23,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,35.0,Oxford/AstraZeneca,Government of Afghanistan,http://www.xinhuanet.com/english/asiapacific/2...
2,Afghanistan,AFG,2021-02-24,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,35.0,Oxford/AstraZeneca,Government of Afghanistan,http://www.xinhuanet.com/english/asiapacific/2...
3,Afghanistan,AFG,2021-02-25,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,35.0,Oxford/AstraZeneca,Government of Afghanistan,http://www.xinhuanet.com/english/asiapacific/2...
4,Afghanistan,AFG,2021-02-26,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,35.0,Oxford/AstraZeneca,Government of Afghanistan,http://www.xinhuanet.com/english/asiapacific/2...
5,Afghanistan,AFG,2021-02-27,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,35.0,Oxford/AstraZeneca,Government of Afghanistan,http://www.xinhuanet.com/english/asiapacific/2...
6,Afghanistan,AFG,2021-02-28,8200.0,8200.0,NaN,NaN,1367.0,0.02,0.02,NaN,35.0,Oxford/AstraZeneca,Government of Afghanistan,http://www.xinhuanet.com/english/asiapacific/2...
7,Afghanistan,AFG,2021-03-01,NaN,NaN,NaN,NaN,1580.0,NaN,NaN,NaN,41.0,Oxford/AstraZeneca,Government of Afghanistan,http://www.xinhuanet.com/english/asiapacific/2...
8,Afghanistan,AFG,2021-03-02,NaN,NaN,NaN,NaN,1794.0,NaN,NaN,NaN,46.0,Oxford/AstraZeneca,Government of Afghanistan,http://www.xinhuanet.com/english/asiapacific/2...
9,Afghanistan,AFG,2021-03-03,NaN,NaN,NaN,NaN,2008.0,NaN,NaN,NaN,52.0,Oxford/AstraZeneca,Government of Afghanistan,http://www.xinhuanet.com/english/asiapacific/2...


In [ ]:
df.columns

Index(['country', 'iso_code', 'date', 'total_vaccinations',
       'people_vaccinated', 'people_fully_vaccinated',
       'daily_vaccinations_raw', 'daily_vaccinations',
       'total_vaccinations_per_hundred', 'people_vaccinated_per_hundred',
       'people_fully_vaccinated_per_hundred', 'daily_vaccinations_per_million',
       'vaccines', 'source_name', 'source_website'],
      dtype='object')

In [ ]:
df.shape

(7488, 15)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7488 entries, 0 to 7487
Data columns (total 15 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   country                              7488 non-null   object 
 1   iso_code                             7488 non-null   object 
 2   date                                 7488 non-null   object 
 3   total_vaccinations                   4625 non-null   float64
 4   people_vaccinated                    4118 non-null   float64
 5   people_fully_vaccinated              2794 non-null   float64
 6   daily_vaccinations_raw               3891 non-null   float64
 7   daily_vaccinations                   7301 non-null   float64
 8   total_vaccinations_per_hundred       4625 non-null   float64
 9   people_vaccinated_per_hundred        4118 non-null   float64
 10  people_fully_vaccinated_per_hundred  2794 non-null   float64
 11  daily_vaccinations_per_million

#Daftar Negara

In [ ]:
df.country.value_counts()


Northern Ireland     97
Wales                97
England              97
United Kingdom       97
Scotland             97
                     ..
Malawi                3
Bahamas               1
Guinea                1
Equatorial Guinea     1
Laos                  1
Name: country, Length: 150, dtype: int64

#Daftar Vaksin

In [ ]:
df.vaccines.value_counts()

Moderna, Oxford/AstraZeneca, Pfizer/BioNTech                                          1813
Pfizer/BioNTech                                                                       1290
Oxford/AstraZeneca, Pfizer/BioNTech                                                    970
Oxford/AstraZeneca                                                                     883
Moderna, Pfizer/BioNTech                                                               385
Sputnik V                                                                              278
Sinovac                                                                                211
Pfizer/BioNTech, Sinovac                                                               204
Oxford/AstraZeneca, Pfizer/BioNTech, Sinopharm/Beijing, Sputnik V                      160
Sinopharm/Beijing                                                                      145
Oxford/AstraZeneca, Sinopharm/Beijing, Sputnik V                                       123

In [ ]:
df_new = df.drop(columns=[
                          'iso_code', 'total_vaccinations', 'people_vaccinated', 
                          'people_fully_vaccinated', 'daily_vaccinations_raw', 
                          'daily_vaccinations', 'total_vaccinations_per_hundred', 
                          'people_vaccinated_per_hundred', 
                          'people_fully_vaccinated_per_hundred', 
                          'daily_vaccinations_per_million'
                          ])
df_new

,country,date,vaccines,source_name,source_website
0,Afghanistan,2021-02-22,Oxford/AstraZeneca,Government of Afghanistan,http://www.xinhuanet.com/english/asiapacific/2...
1,Afghanistan,2021-02-23,Oxford/AstraZeneca,Government of Afghanistan,http://www.xinhuanet.com/english/asiapacific/2...
2,Afghanistan,2021-02-24,Oxford/AstraZeneca,Government of Afghanistan,http://www.xinhuanet.com/english/asiapacific/2...
3,Afghanistan,2021-02-25,Oxford/AstraZeneca,Government of Afghanistan,http://www.xinhuanet.com/english/asiapacific/2...
4,Afghanistan,2021-02-26,Oxford/AstraZeneca,Government of Afghanistan,http://www.xinhuanet.com/english/asiapacific/2...
...,...,...,...,...,...
7483,Zimbabwe,2021-03-16,Sinopharm/Beijing,Ministry of Health,https://twitter.com/MoHCCZim/status/1373390070...
7484,Zimbabwe,2021-03-17,Sinopharm/Beijing,Ministry of Health,https://twitter.com/MoHCCZim/status/1373390070...
7485,Zimbabwe,2021-03-18,Sinopharm/Beijing,Ministry of Health,https://twitter.com/MoHCCZim/status/1373390070...
7486,Zimbabwe,2021-03-19,Sinopharm/Beijing,Ministry of Health,https://twitter.com/MoHCCZim/status/1373390070...


In [ ]:
# import and download package
import nltk, os, re, string

from keras.layers import Input, LSTM, Bidirectional, SpatialDropout1D, Dropout, Flatten, Dense, Embedding, BatchNormalization
from keras.models import Model
from keras.callbacks import EarlyStopping
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet as wn

# lower-case all characters
df_new.country = df_new.country.apply(lambda x: x.lower())
df_new.date = df_new.date.apply(lambda x: x.lower())
df_new.vaccines = df_new.vaccines.apply(lambda x: x.lower())
df_new.source_name = df_new.source_name.apply(lambda x: x.lower())
df_new.source_website = df_new.source_website.apply(lambda x: x.lower())


In [ ]:
category = pd.get_dummies(df_new.vaccines)
df_new_cat = pd.concat([df_new, category], axis=1)
df_new_cat = df_new_cat.drop(columns='vaccines')
df_new_cat.head(20)

,country,date,source_name,source_website,"covaxin, oxford/astrazeneca","epivaccorona, sputnik v",johnson&johnson,"johnson&johnson, moderna, pfizer/biontech",moderna,"moderna, oxford/astrazeneca","moderna, oxford/astrazeneca, pfizer/biontech","moderna, oxford/astrazeneca, pfizer/biontech, sinopharm/beijing, sputnik v","moderna, pfizer/biontech",oxford/astrazeneca,"oxford/astrazeneca, pfizer/biontech","oxford/astrazeneca, pfizer/biontech, sinopharm/beijing, sinopharm/wuhan, sputnik v","oxford/astrazeneca, pfizer/biontech, sinopharm/beijing, sputnik v","oxford/astrazeneca, pfizer/biontech, sputnik v","oxford/astrazeneca, sinopharm/beijing","oxford/astrazeneca, sinopharm/beijing, sputnik v","oxford/astrazeneca, sinovac",pfizer/biontech,"pfizer/biontech, sinopharm/beijing","pfizer/biontech, sinovac",sinopharm/beijing,"sinopharm/beijing, sinopharm/wuhan, sinovac","sinopharm/beijing, sputnik v",sinovac,sputnik v
0,afghanistan,2021-02-22,government of afghanistan,http://www.xinhuanet.com/english/asiapacific/2...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,afghanistan,2021-02-23,government of afghanistan,http://www.xinhuanet.com/english/asiapacific/2...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,afghanistan,2021-02-24,government of afghanistan,http://www.xinhuanet.com/english/asiapacific/2...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,afghanistan,2021-02-25,government of afghanistan,http://www.xinhuanet.com/english/asiapacific/2...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,afghanistan,2021-02-26,government of afghanistan,http://www.xinhuanet.com/english/asiapacific/2...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,afghanistan,2021-02-27,government of afghanistan,http://www.xinhuanet.com/english/asiapacific/2...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,afghanistan,2021-02-28,government of afghanistan,http://www.xinhuanet.com/english/asiapacific/2...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,afghanistan,2021-03-01,government of afghanistan,http://www.xinhuanet.com/english/asiapacific/2...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,afghanistan,2021-03-02,government of afghanistan,http://www.xinhuanet.com/english/asiapacific/2...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,afghanistan,2021-03-03,government of afghanistan,http://www.xinhuanet.com/english/asiapacific/2...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
covid = df_new_cat['country'].values + '' + df_new_cat['date'].values + '' + df_new_cat['source_name'].values  + '' + df_new_cat['source_website'].values
label = df_new_cat[lambda nama: nama.lower(['Covaxin, Oxford/AstraZeneca', 'EpiVacCorona, Sputnik V', 	
                    'Johnson&Johnson', 'Johnson&Johnson, Moderna, Pfizer/BioNTech', 	
                    'Moderna', 'Moderna, Oxford/AstraZeneca', 'Moderna, Oxford/AstraZeneca, Pfizer/BioNTech',  
                    'Moderna, Oxford/AstraZeneca, Pfizer/BioNTech, Sinopharm/Beijing, Sputnik V', 	
                    'Moderna, Pfizer/BioNTech', 'Oxford/AstraZeneca', 'Oxford/AstraZeneca, Pfizer/BioNTech',  
                    'Oxford/AstraZeneca, Pfizer/BioNTech, Sinopharm/Beijing, Sinopharm/Wuhan, Sputnik V', 
                    'Oxford/AstraZeneca, Pfizer/BioNTech, Sinopharm/Beijing, Sputnik V',  
                    'Oxford/AstraZeneca, Pfizer/BioNTech, Sputnik V',  
                    'Oxford/AstraZeneca, Sinopharm/Beijing', 'Oxford/AstraZeneca, Sinopharm/Beijing, Sputnik V',  	
                    'Oxford/AstraZeneca, Sinovac', 'Pfizer/BioNTech', 'Pfizer/BioNTech, Sinopharm/Beijing', 
                    'Pfizer/BioNTech, Sinovac', 'Sinopharm/Beijing', 
                    'Sinopharm/Beijing, Sinopharm/Wuhan, Sinovac', 
                    'Sinopharm/Beijing, Sputnik V', 'Sinovac', 'Sputnik V'])].values

AttributeError: ignored

In [ ]:
from sklearn.model_selection import train_test_split
covid_train, covid_test, label_train, label_test = train_test_split(covid, label, test_size=0.2, shuffle=True)